In [3]:
import pandas as pd
import os
import datetime as dt
pd.set_option('display.max_columns',100)
import matplotlib.pyplot as plt
import seaborn as sns
# !pip install koreanize-matplotlib
import koreanize_matplotlib
import matplotlib as mpl
pd.set_option('display.max_colwidth', None)
import glob

### enter_lesson_page

#### 기본 전처리 ()

In [4]:
enter_lesson_page = pd.read_csv('C:/Users/syc09/Desktop/코드잇/중급프로젝트1/Topic2_real/enter_lesson_page-001.csv')

C:\Users\syc09\AppData\Local\Temp\ipykernel_17992\2524196629.py:1: DtypeWarning: Columns (9,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  enter_lesson_page = pd.read_csv('C:/Users/syc09/Desktop/코드잇/중급프로젝트1/Topic2_real/enter_lesson_page-001.csv')


In [5]:
# 시간. UTC -> KST
enter_lesson_page['client_event_time'] = pd.to_datetime(enter_lesson_page['client_event_time'], errors='coerce')
enter_lesson_page['client_event_time'] = enter_lesson_page['client_event_time'] + pd.Timedelta(hours=9)

In [6]:
# 2022, 2023년 제한
enter_lesson_page = enter_lesson_page.loc[
    enter_lesson_page['client_event_time'].dt.year.between(2022, 2023)]

In [7]:
# 컬럼명 변경
enter_lesson_page.rename(columns={'content.id': 'content_id', 'lesson.id': 'lesson_id'}, inplace=True)
enter_lesson_page.rename(columns={'client_event_time': 'enter_lesson_time'}, inplace=True)

# 컬럼 버리기
enter_lesson_page.drop(columns=['platform', 'os_version', 'device_carrier'], inplace=True)
enter_lesson_page.drop(enter_lesson_page[enter_lesson_page['os_name'] == 'Chrome Headless'].index, inplace=True)

In [8]:
# 중복 컬럼 삭제
enter_lesson_page.duplicated(subset=['enter_lesson_time', 'user_id', 'content_id', 'lesson_id'], keep=False)

0           False
1           False
2           False
3           False
4           False
            ...  
21029702    False
21029703    False
21029704    False
21029705    False
21029706    False
Length: 21028196, dtype: bool

In [9]:
enter_lesson_page.head()

,city,enter_lesson_time,country,device_family,device_type,event_type,language,os_name,user_id,content_id,is_trial,lesson_id,is_free_trial
0,Wonju,2023-04-06 06:46:04.333,South Korea,Windows,Windows,enter.lesson_page,Korean,Chrome,91053eda0bffb4db6c9317acf15bc44a,94bdcbd9f329aafa84ab464b6721187d,False,d360d68b65fd3e0eafb7982386d75fd9,NaN
1,Wonju,2023-04-06 06:55:50.787,South Korea,Windows,Windows,enter.lesson_page,Korean,Chrome,91053eda0bffb4db6c9317acf15bc44a,94bdcbd9f329aafa84ab464b6721187d,False,d360d68b65fd3e0eafb7982386d75fd9,NaN
2,Wonju,2023-04-06 06:55:55.684,South Korea,Windows,Windows,enter.lesson_page,Korean,Chrome,91053eda0bffb4db6c9317acf15bc44a,94bdcbd9f329aafa84ab464b6721187d,False,770740696ab1809d6b50d4013b64a75c,NaN
3,Dongdaemun-gu,2023-04-06 06:04:55.828,South Korea,Mac,Mac,enter.lesson_page,Korean,Chrome,b2b31f00b82edc6419b25324a7ae41a4,5fc64d786416b980cfda86afd69e4516,False,68cc362ca81b5899b3c9b4dea0aaddff,NaN
4,Dongdaemun-gu,2023-04-06 06:05:34.139,South Korea,Mac,Mac,enter.lesson_page,Korean,Chrome,b2b31f00b82edc6419b25324a7ae41a4,db43a841c994231e2795d4df8931af50,False,144682f4648f81de8e6aabd11b92105e,NaN


### complete_lesson

#### 기본 전처리

In [10]:
complete_lesson = pd.read_csv('C:/Users/syc09/Desktop/코드잇/중급프로젝트1/Topic2_real/complete_lesson.csv')

In [11]:
# 시간. UTC -> KST
complete_lesson['client_event_time'] = pd.to_datetime(complete_lesson['client_event_time'], errors='coerce')
complete_lesson['client_event_time'] = complete_lesson['client_event_time'] + pd.Timedelta(hours=9)

In [12]:
# 2022, 2023년 제한
# complete_lesson = complete_lesson[(complete_lesson['client_event_time'] >= start_date) & (complete_lesson['client_event_time'] <= end_date)]
# 시간 필터링
complete_lesson = complete_lesson.loc[
    complete_lesson['client_event_time'].dt.year.between(2022, 2023)]

In [13]:
# 컬럼명 변경
complete_lesson.rename(columns={'content.id': 'content_id', 'lesson.id': 'lesson_id'}, inplace=True)
complete_lesson.rename(columns={'client_event_time': 'complete_lesson_time'}, inplace=True)

# 컬럼 버리기
complete_lesson.drop(columns=['platform', 'os_version', 'device_carrier'], inplace=True)
complete_lesson.drop(complete_lesson[complete_lesson['os_name'] == 'Chrome Headless'].index, inplace=True)

In [14]:
# 중복 컬럼 삭제
complete_lesson.duplicated(subset=['complete_lesson_time', 'user_id', 'content_id', 'lesson_id'], keep=False)

0          False
1          False
2          False
3          False
4          False
           ...  
5405064    False
5405065    False
5405066    False
5405067    False
5405068    False
Length: 5404242, dtype: bool

In [15]:
complete_lesson.head()

,city,complete_lesson_time,country,device_family,device_type,event_type,language,os_name,user_id,content_id,lesson_id
0,Wonju,2023-04-06 06:55:51.836,South Korea,Windows,Windows,complete.lesson,Korean,Chrome,91053eda0bffb4db6c9317acf15bc44a,94bdcbd9f329aafa84ab464b6721187d,d360d68b65fd3e0eafb7982386d75fd9
1,Daejeon,2023-04-06 06:19:01.243,South Korea,Windows,Windows,complete.lesson,Korean,Edge,41362ad5ebcade2bb1b78344a53e7ccf,445fac33232adcb3d1cdab7d8a579430,7e0f6aae886368ef9fa97007a8e9f7ac
2,Daejeon,2023-04-06 06:20:27.939,South Korea,Windows,Windows,complete.lesson,Korean,Edge,41362ad5ebcade2bb1b78344a53e7ccf,445fac33232adcb3d1cdab7d8a579430,d1e93c9dc08267efab31849bc6083854
3,Daejeon,2023-04-06 06:23:01.534,South Korea,Windows,Windows,complete.lesson,Korean,Edge,41362ad5ebcade2bb1b78344a53e7ccf,445fac33232adcb3d1cdab7d8a579430,f22ea5dc431035305e705552bbc4f45b
4,Daejeon,2023-04-06 06:28:34.965,South Korea,Windows,Windows,complete.lesson,Korean,Edge,41362ad5ebcade2bb1b78344a53e7ccf,445fac33232adcb3d1cdab7d8a579430,ce0bfeb7fb1259c588f437d48ab2690c


In [16]:
print(complete_lesson['complete_lesson_time'].min())
print(complete_lesson['complete_lesson_time'].max())

2022-01-01 09:00:01.181000
2023-12-31 23:59:51.635000


### 각 데이터에서 30초 이내 로그가 찍힌 것들, 동일한 년, 월, 일, 시, 분 삭제(같은 레슨일 경우)

- enter lesson

In [17]:
# enter_lesson_time을 시간 순서대로 정렬
enter_lesson_page_filtered = enter_lesson_page.sort_values(by=['user_id', 'enter_lesson_time'])

# 동일한 user_id에 대해 연속된 행의 시간 차이 계산
enter_lesson_page_filtered['time_diff'] = enter_lesson_page_filtered.groupby('user_id')['enter_lesson_time'].diff()

# 시간 차이를 초 단위로 변환
enter_lesson_page_filtered['time_diff_seconds'] = enter_lesson_page_filtered['time_diff'].dt.total_seconds() #.round(2)

In [18]:
# time_diff_seconds가 30초 초과인 데이터만 남김김
enter_lesson_page_finalised = enter_lesson_page_filtered[(enter_lesson_page_filtered['time_diff_seconds'] > 30)]

In [19]:
# 년-월-일-시-분 단위 컬럼 추가
enter_lesson_page_finalised["time_group"] = enter_lesson_page_finalised["enter_lesson_time"].dt.strftime("%Y-%m-%d %H:%M")

# 정렬 수행
enter_lesson_page_finalised = enter_lesson_page_finalised.sort_values(by=["user_id", "content_id", "lesson_id", "enter_lesson_time"])

# 중복 제거 (첫 번째 값 유지)
enter_lesson_page_finalised_unique = enter_lesson_page_finalised.drop_duplicates(subset=["user_id", "content_id", "lesson_id", "time_group"], keep="first")

enter_lesson_page_finalised_unique = enter_lesson_page_finalised_unique.drop_duplicates(
    subset=["user_id", "time_group"], keep="first"
)


# 필요 없는 컬럼 제거
enter_lesson_page_finalised_unique = enter_lesson_page_finalised_unique.drop(columns=["time_group"])


C:\Users\syc09\AppData\Local\Temp\ipykernel_17992\4201793937.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enter_lesson_page_finalised["time_group"] = enter_lesson_page_finalised["enter_lesson_time"].dt.strftime("%Y-%m-%d %H:%M")


In [20]:
print(len(enter_lesson_page)) # 로우 데이터
print(len(enter_lesson_page_finalised)) # timediff 30초 초과 데이터
print(len(enter_lesson_page_finalised_unique)) # 동일한 년월일시분 삭제

21028196
8118190
7852494


In [21]:
print(enter_lesson_page['user_id'].nunique()) # 로우 데이터
print(enter_lesson_page_finalised['user_id'].nunique()) # timediff 30초 초과 데이터
print(enter_lesson_page_finalised_unique['user_id'].nunique()) # 동일한 년월일시분 삭제

113916
79820
79820


In [22]:
print(enter_lesson_page['lesson_id'].nunique()) # 로우 데이터
print(enter_lesson_page_finalised['lesson_id'].nunique()) # timediff 30초 초과 데이터
print(enter_lesson_page_finalised_unique['lesson_id'].nunique()) # 동일한 년월일시분 삭제

5479
5391
5383


In [23]:
enter_lesson_page_finalised_unique.describe()

,enter_lesson_time,time_diff,time_diff_seconds
count,7852494,7852494,7.852494e+06
mean,2023-01-07 20:17:21.373091584,0 days 15:31:14.171259363,5.587417e+04
min,2022-01-01 00:56:27.589000,0 days 00:00:30.001000,3.000100e+01
25%,2022-07-24 19:21:33.735750144,0 days 00:01:25.004250,8.500425e+01
50%,2023-01-26 11:07:25.428499968,0 days 00:03:20.362000,2.003620e+02
75%,2023-06-30 00:36:46.983249920,0 days 00:09:22.805000,5.628050e+02
max,2023-12-31 23:59:45.192000,722 days 23:20:52.788000,6.246485e+07
std,NaN,9 days 22:08:57.636050669,8.573376e+05


#### complete lesson

In [24]:
# enter_lesson_time을 시간 순서대로 정렬
complete_lesson_filtered = complete_lesson.sort_values(by=['user_id', 'complete_lesson_time'])

# 동일한 user_id에 대해 연속된 행의 시간 차이 계산
complete_lesson_filtered['time_diff'] = complete_lesson_filtered.groupby('user_id')['complete_lesson_time'].diff()

# 시간 차이를 초 단위로 변환
complete_lesson_filtered['time_diff_seconds'] = complete_lesson_filtered['time_diff'].dt.total_seconds() #.round(2)

In [25]:
# time_diff_seconds가 30초 초과인 데이터만 남김김
complete_lesson_finalised = complete_lesson_filtered[(complete_lesson_filtered['time_diff_seconds'] > 30)]

In [26]:
# 년-월-일-시-분 단위 컬럼 추가
complete_lesson_finalised["time_group"] = complete_lesson_finalised["complete_lesson_time"].dt.strftime("%Y-%m-%d %H:%M")

# 정렬 수행
complete_lesson_finalised = complete_lesson_finalised.sort_values(by=["user_id", "content_id", "lesson_id", "complete_lesson_time"])

# 중복 제거 (첫 번째 값 유지)
complete_lesson_finalised_unique = complete_lesson_finalised.drop_duplicates(subset=["user_id", "content_id", "lesson_id", "time_group"], keep="first")

complete_lesson_finalised_unique = complete_lesson_finalised_unique.drop_duplicates(
    subset=["user_id", "time_group"], keep="first"
)

# 필요 없는 컬럼 제거
complete_lesson_finalised_unique = complete_lesson_finalised_unique.drop(columns=["time_group"])


C:\Users\syc09\AppData\Local\Temp\ipykernel_17992\3935886420.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete_lesson_finalised["time_group"] = complete_lesson_finalised["complete_lesson_time"].dt.strftime("%Y-%m-%d %H:%M")


In [27]:
complete_lesson_finalised_unique.head()

,city,complete_lesson_time,country,device_family,device_type,event_type,language,os_name,user_id,content_id,lesson_id,time_diff,time_diff_seconds
222221,Seoul,2023-03-19 01:29:04.507,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,1e2a48ce24bce72e7d36da219ee9d254,0 days 00:05:53.192000,353.192
222220,Seoul,2023-03-19 01:23:11.315,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,3ae4d4504b620370687e58e0a33cf22d,0 days 00:04:43.050000,283.050
1199632,Seoul,2023-03-19 00:53:48.730,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,4d65e0729706704bf34953f2613d6ecd,0 days 00:02:14.017000,134.017
222225,Seoul,2023-03-19 01:48:55.626,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,58a55a039ef52567cfe0821f92b3c9e6,0 days 00:06:13.406000,373.406
222224,Seoul,2023-03-19 01:42:42.220,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,adfe4e8b1dc5d9f2e4b76e4bc8ee221c,0 days 00:04:11.322000,251.322


In [28]:
print(len(complete_lesson)) # 로우 데이터
print(len(complete_lesson_finalised)) # timediff 30초 초과 데이터
print(len(complete_lesson_finalised_unique)) # 동일한 년월일시분 삭제

5404242
4502312
4427700


In [29]:
print(complete_lesson['user_id'].nunique()) # 로우 데이터
print(complete_lesson_finalised['user_id'].nunique()) # timediff 30초 초과 데이터
print(complete_lesson_finalised_unique['user_id'].nunique()) # 동일한 년월일시분 삭제

49670
44419
44419


In [30]:
print(complete_lesson['lesson_id'].nunique()) # 로우 데이터
print(complete_lesson_finalised['lesson_id'].nunique()) # timediff 30초 초과 데이터
print(complete_lesson_finalised_unique['lesson_id'].nunique()) # 동일한 년월일시분 삭제

5050
5036
5034


In [31]:
complete_lesson_finalised_unique.describe()

,complete_lesson_time,time_diff,time_diff_seconds
count,4427700,4427700,4.427700e+06
mean,2023-01-21 05:03:18.948490752,0 days 15:37:22.346357195,5.624235e+04
min,2022-01-01 09:06:56.209000,0 days 00:00:30.001000,3.000100e+01
25%,2022-07-25 15:36:46.370500096,0 days 00:02:23.719000,1.437190e+02
50%,2023-01-28 16:36:47.536499968,0 days 00:04:56.299000,2.962990e+02
75%,2023-07-31 16:20:14.356499968,0 days 00:14:59.997250,8.999973e+02
max,2023-12-31 23:59:51.635000,720 days 13:46:13.325000,6.225757e+07
std,NaN,8 days 11:20:42.775909348,7.320428e+05


In [32]:
complete_lesson_finalised.describe()

,complete_lesson_time,time_diff,time_diff_seconds
count,4502312,4502312,4.502312e+06
mean,2023-01-21 03:30:51.147414016,0 days 15:28:03.565471665,5.568357e+04
min,2022-01-01 09:06:56.209000,0 days 00:00:30.001000,3.000100e+01
25%,2022-07-25 15:50:16.603500032,0 days 00:02:20.210000,1.402100e+02
50%,2023-01-28 13:08:16.047500032,0 days 00:04:51.706500,2.917065e+02
75%,2023-07-31 15:27:12.669250048,0 days 00:14:41.486500,8.814865e+02
max,2023-12-31 23:59:51.635000,720 days 13:46:13.325000,6.225757e+07
std,NaN,8 days 10:21:22.877575034,7.284829e+05


- complete_subscription에서 user_id를 가져오기

In [33]:
# complete_subscription['complete_subscription_time'].max()

In [34]:
# complete_subscription.isna().sum()

In [35]:
# 파일 불러오기 및 전처리리
complete_subscription = pd.read_csv('C:/Users/syc09/Desktop/코드잇/중급프로젝트1/Topic2_real/complete_subscription.csv')
complete_subscription['client_event_time'] = pd.to_datetime(complete_subscription['client_event_time'], errors='coerce')
complete_subscription['client_event_time'] = complete_subscription['client_event_time'] + pd.Timedelta(hours=9)
complete_subscription.drop(complete_subscription[complete_subscription['os_name'] == 'Chrome Headless'].index, inplace=True)


# 2022, 2023년 제한
# 시간 필터링
complete_subscription = complete_subscription.loc[
    complete_subscription['client_event_time'].dt.year.between(2022, 2023)]

In [36]:
complete_subscription = complete_subscription.drop_duplicates(subset=['user_id'], keep='first')
complete_subscription['user_id'].nunique()

13875

In [37]:
complete_subscription['client_event_time'].max()

Timestamp('2023-12-31 23:59:39.461000')

In [38]:
complete_subscription.isna().sum()

city                        522
client_event_time             0
country                     376
device_carrier            13875
device_family               368
device_type                 517
event_type                    0
language                    368
os_name                     374
os_version                  383
platform                    368
user_id                       0
plan.price                    0
paid_amount                   0
coupon.discount_amount        0
pg.type                   12274
dtype: int64

In [39]:
complete_subscription['user_id'].dropna().unique()

array(['1116a2ae90ca6b06f85feba27ad51ef0',
       'c0f5678df8967163e6e2c464ed006791',
       '79421b5d038205ad7adbb1e2c3b89741', ...,
       'abebe3234041d14b5c0c5821bf90ab26',
       '3ca22b0bade6b40d6274bf6d06bc4344',
       'c49ea18247c81a613f8bbf44ff029cb4'], dtype=object)

In [40]:
# 중복되는 user_id 제거
complete_subscription = complete_subscription.drop_duplicates(subset=['user_id'], keep='first')
complete_subscription['user_id'].nunique()

13875

In [41]:
# complete_subscription 파일에서 user_id 컬럼만 추출하여 저장
subscription_user_id_list = complete_subscription['user_id'].unique()

In [42]:
# subscription_user_id_list의 데이터 타입 확인
subscription_user_id_list.dtype

dtype('O')

In [43]:
subscription_user_id_list

array(['1116a2ae90ca6b06f85feba27ad51ef0',
       'c0f5678df8967163e6e2c464ed006791',
       '79421b5d038205ad7adbb1e2c3b89741', ...,
       'abebe3234041d14b5c0c5821bf90ab26',
       '3ca22b0bade6b40d6274bf6d06bc4344',
       'c49ea18247c81a613f8bbf44ff029cb4'], dtype=object)

#### enter_lesson, complete_lesson에 subscription_user_id_list의 user_id 필터링

- enter_lesson

In [44]:
# enter_lesson_page_finalised_unique 파일에서 subscription_user_id_list에 있는 user_id만 추출하여 저장
enter_lesson_page_finalised_unique_sub = enter_lesson_page_finalised_unique[enter_lesson_page_finalised_unique['user_id'].isin(subscription_user_id_list)]
enter_lesson_page_finalised_unique_sub.head()

,city,enter_lesson_time,country,device_family,device_type,event_type,language,os_name,user_id,content_id,is_trial,lesson_id,is_free_trial,time_diff,time_diff_seconds
16138,Seoul,2023-03-18 19:42:48.654,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,True,09c8a2d29087cb3d47823acb5f2705e3,NaN,0 days 00:05:42.876000,342.876
6147610,Seoul,2023-03-19 00:46:47.244,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,False,0cf175a1781a8e0020bf6c6bdc80e78a,NaN,0 days 00:01:11.785000,71.785
6147611,Seoul,2023-03-19 00:51:34.741,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,False,0cf175a1781a8e0020bf6c6bdc80e78a,NaN,0 days 00:04:47.497000,287.497
1140249,Seoul,2023-03-19 01:29:04.118,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,False,1e2a48ce24bce72e7d36da219ee9d254,NaN,0 days 00:05:50.893000,350.893
6147608,Seoul,2023-03-19 00:45:24.704,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,False,2ed4b83e65edc2b1efb193aaa3c60cf9,NaN,0 days 01:12:02.068000,4322.068


In [45]:
print(len(enter_lesson_page_finalised_unique_sub))
print(enter_lesson_page_finalised_unique_sub['user_id'].nunique())

3389303
12860


In [46]:
# complete_lesson_finalised_unique 파일에서 subscription_user_id_list에 있는 user_id만 추출하여 저장
complete_lesson_finalised_unique_sub = complete_lesson_finalised_unique[complete_lesson_finalised_unique['user_id'].isin(subscription_user_id_list)]
complete_lesson_finalised_unique_sub.head()

,city,complete_lesson_time,country,device_family,device_type,event_type,language,os_name,user_id,content_id,lesson_id,time_diff,time_diff_seconds
222221,Seoul,2023-03-19 01:29:04.507,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,1e2a48ce24bce72e7d36da219ee9d254,0 days 00:05:53.192000,353.192
222220,Seoul,2023-03-19 01:23:11.315,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,3ae4d4504b620370687e58e0a33cf22d,0 days 00:04:43.050000,283.050
1199632,Seoul,2023-03-19 00:53:48.730,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,4d65e0729706704bf34953f2613d6ecd,0 days 00:02:14.017000,134.017
222225,Seoul,2023-03-19 01:48:55.626,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,58a55a039ef52567cfe0821f92b3c9e6,0 days 00:06:13.406000,373.406
222224,Seoul,2023-03-19 01:42:42.220,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,adfe4e8b1dc5d9f2e4b76e4bc8ee221c,0 days 00:04:11.322000,251.322


In [47]:
print(len(complete_lesson_finalised_unique_sub))
print(complete_lesson_finalised_unique_sub['user_id'].nunique())

1860818
12317


- enter_lesson_page

In [48]:
#  첫 결제 유저 ID 목록 (중복 제거 후 리스트로 변환)
unique_first_payment_user_ids = complete_subscription['user_id'].dropna().unique().tolist()

# enter_lesson_page_finalised_unique에서 첫 결제 유저만 필터링
first_paid_enter_lesson_page_finalised_unique = enter_lesson_page_finalised_unique[enter_lesson_page_finalised_unique['user_id'].isin(unique_first_payment_user_ids)]

# 결과 출력
print("첫 결제 유저 필터링 개수:", len(first_paid_enter_lesson_page_finalised_unique))

첫 결제 유저 필터링 개수: 3389303


- complete_lesson

In [49]:
#  첫 결제 유저 ID 목록 (중복 제거 후 리스트로 변환)
unique_first_payment_user_ids = complete_subscription['user_id'].dropna().unique().tolist()

# complete_lesson_finalised_unique에서 첫 결제 유저만 필터링
first_paid_complete_lesson_finalised_unique = complete_lesson_finalised_unique[complete_lesson_finalised_unique['user_id'].isin(unique_first_payment_user_ids)]

# 결과 출력
print("첫 결제 유저 필터링 개수:", len(first_paid_complete_lesson_finalised_unique))

첫 결제 유저 필터링 개수: 1860818


In [50]:
first_paid_enter_lesson_page_finalised_unique.head()

,city,enter_lesson_time,country,device_family,device_type,event_type,language,os_name,user_id,content_id,is_trial,lesson_id,is_free_trial,time_diff,time_diff_seconds
16138,Seoul,2023-03-18 19:42:48.654,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,True,09c8a2d29087cb3d47823acb5f2705e3,NaN,0 days 00:05:42.876000,342.876
6147610,Seoul,2023-03-19 00:46:47.244,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,False,0cf175a1781a8e0020bf6c6bdc80e78a,NaN,0 days 00:01:11.785000,71.785
6147611,Seoul,2023-03-19 00:51:34.741,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,False,0cf175a1781a8e0020bf6c6bdc80e78a,NaN,0 days 00:04:47.497000,287.497
1140249,Seoul,2023-03-19 01:29:04.118,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,False,1e2a48ce24bce72e7d36da219ee9d254,NaN,0 days 00:05:50.893000,350.893
6147608,Seoul,2023-03-19 00:45:24.704,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,False,2ed4b83e65edc2b1efb193aaa3c60cf9,NaN,0 days 01:12:02.068000,4322.068


In [53]:
first_paid_complete_lesson_finalised_unique.head()

,city,complete_lesson_time,country,device_family,device_type,event_type,language,os_name,user_id,content_id,lesson_id,time_diff,time_diff_seconds
222221,Seoul,2023-03-19 01:29:04.507,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,1e2a48ce24bce72e7d36da219ee9d254,0 days 00:05:53.192000,353.192
222220,Seoul,2023-03-19 01:23:11.315,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,3ae4d4504b620370687e58e0a33cf22d,0 days 00:04:43.050000,283.050
1199632,Seoul,2023-03-19 00:53:48.730,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,4d65e0729706704bf34953f2613d6ecd,0 days 00:02:14.017000,134.017
222225,Seoul,2023-03-19 01:48:55.626,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,58a55a039ef52567cfe0821f92b3c9e6,0 days 00:06:13.406000,373.406
222224,Seoul,2023-03-19 01:42:42.220,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,adfe4e8b1dc5d9f2e4b76e4bc8ee221c,0 days 00:04:11.322000,251.322


#### 원찬님 방법

In [54]:
# # 임시 리스트 생성
# temp_list = []

# for data in [enter_lesson_page_df, complete_lesson_df]:
#     # 필요한 컬럼만 추출하여 필터링
#     data = data[['user_id', 'client_event_time', 'event_type', 'content.id', 'lesson.id']]

#     # 임시 리스트에 저장
#     temp_list.append(data)

# # 임시 리스트를 사용하여 데이터 concat
# lesson_concat_df = pd.concat(temp_list, axis=0)

# # reset_index()로 인덱스 재정렬
# lesson_concat_df.reset_index(drop=True)

# # user_id, client_event_time을 기준으로 오름차순 정렬
# lesson_concat_df.sort_values(by=['user_id', 'client_event_time'], ascending=[True, True])

# # 데이터 확인
# lesson_concat_df

### concat으로 lesson 수강 시간 등을 알아보자

In [55]:
# 데이터의 lesson_time 컬럼 이름을 event_time으로 변경
first_paid_enter_lesson_page_finalised_unique.rename(columns={'enter_lesson_time': 'event_time'}, inplace=True)
first_paid_complete_lesson_finalised_unique.rename(columns={'complete_lesson_time': 'event_time'}, inplace=True)

C:\Users\syc09\AppData\Local\Temp\ipykernel_17992\1196140547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_paid_enter_lesson_page_finalised_unique.rename(columns={'enter_lesson_time': 'event_time'}, inplace=True)
C:\Users\syc09\AppData\Local\Temp\ipykernel_17992\1196140547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_paid_complete_lesson_finalised_unique.rename(columns={'complete_lesson_time': 'event_time'}, inplace=True)


In [59]:
# 임시 리스트 생성
temp_list = []

for data in [first_paid_enter_lesson_page_finalised_unique, first_paid_complete_lesson_finalised_unique]:
    # 필요한 컬럼만 추출하여 필터링
    data = data[['user_id', 'event_time', 'event_type', 'content_id', 'lesson_id']]

    # 임시 리스트에 저장
    temp_list.append(data)

# concat 제발발
god_please = pd.concat(temp_list, axis=0)

god_please = god_please.reset_index(drop=True)
god_please

,user_id,event_time,event_type,content_id,lesson_id
0,0000572f2ba8079bcefd2760b632e820,2023-03-18 19:42:48.654,enter.lesson_page,61b6463287573f00de13a930805a52d6,09c8a2d29087cb3d47823acb5f2705e3
1,0000572f2ba8079bcefd2760b632e820,2023-03-19 00:46:47.244,enter.lesson_page,61b6463287573f00de13a930805a52d6,0cf175a1781a8e0020bf6c6bdc80e78a
2,0000572f2ba8079bcefd2760b632e820,2023-03-19 00:51:34.741,enter.lesson_page,61b6463287573f00de13a930805a52d6,0cf175a1781a8e0020bf6c6bdc80e78a
3,0000572f2ba8079bcefd2760b632e820,2023-03-19 01:29:04.118,enter.lesson_page,61b6463287573f00de13a930805a52d6,1e2a48ce24bce72e7d36da219ee9d254
4,0000572f2ba8079bcefd2760b632e820,2023-03-19 00:45:24.704,enter.lesson_page,61b6463287573f00de13a930805a52d6,2ed4b83e65edc2b1efb193aaa3c60cf9
...,...,...,...,...,...
5250116,fffadab35ddc8234e1c5139815bfc017,2023-09-11 16:57:14.490,complete.lesson,f4be1a0fe0e301b03115591777fddf29,b05e0fb8ab15def4b33878b9a759b4f2
5250117,fffadab35ddc8234e1c5139815bfc017,2023-09-11 17:05:08.623,complete.lesson,f4be1a0fe0e301b03115591777fddf29,b26325925f88bef76397eedfe8a131c2
5250118,fffadab35ddc8234e1c5139815bfc017,2023-09-11 16:38:37.265,complete.lesson,f4be1a0fe0e301b03115591777fddf29,bfb6543cc0e7fa189f9c9a0e533b0f43
5250119,fffadab35ddc8234e1c5139815bfc017,2023-09-11 17:01:13.881,complete.lesson,f4be1a0fe0e301b03115591777fddf29,cdd77588e214e49bff41b0e7b20b5bb4


In [60]:
# event_type 이름 바꾸기
god_please['event_type'] = god_please['event_type'].replace({
    'enter.lesson_page': 'enter',
    'complete.lesson': 'complete'
})

In [61]:
god_please.head()

,user_id,event_time,event_type,content_id,lesson_id
0,0000572f2ba8079bcefd2760b632e820,2023-03-18 19:42:48.654,enter,61b6463287573f00de13a930805a52d6,09c8a2d29087cb3d47823acb5f2705e3
1,0000572f2ba8079bcefd2760b632e820,2023-03-19 00:46:47.244,enter,61b6463287573f00de13a930805a52d6,0cf175a1781a8e0020bf6c6bdc80e78a
2,0000572f2ba8079bcefd2760b632e820,2023-03-19 00:51:34.741,enter,61b6463287573f00de13a930805a52d6,0cf175a1781a8e0020bf6c6bdc80e78a
3,0000572f2ba8079bcefd2760b632e820,2023-03-19 01:29:04.118,enter,61b6463287573f00de13a930805a52d6,1e2a48ce24bce72e7d36da219ee9d254
4,0000572f2ba8079bcefd2760b632e820,2023-03-19 00:45:24.704,enter,61b6463287573f00de13a930805a52d6,2ed4b83e65edc2b1efb193aaa3c60cf9


In [64]:
god_please.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5250121 entries, 0 to 5250120
Data columns (total 5 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     object        
 1   event_time  datetime64[ns]
 2   event_type  object        
 3   content_id  object        
 4   lesson_id   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 200.3+ MB


In [70]:
# enter, complete 분리
enter_df = god_please[god_please['event_type'] == 'enter']
complete_df = god_please[god_please['event_type'] == 'complete']

# **각각 정렬 (user_id, content_id, lesson_id, event_time 기준)**
enter_df = enter_df.sort_values(['user_id', 'content_id', 'lesson_id', 'event_time'])
enter_df = enter_df.sort_values('event_time', ignore_index=True)
complete_df = complete_df.sort_values(['user_id', 'content_id', 'lesson_id', 'event_time'])
complete_df = complete_df.sort_values('event_time', ignore_index=True)

# 정렬 확인 (디버깅용)
print("enter_df 확인:")
print(enter_df)
print("\ncomplete_df 확인:")
print(complete_df)

matched_df = pd.merge_asof(
    enter_df,
    complete_df.rename(columns={'event_time': 'event_time_complete'}),  # complete의 event_time 컬럼명 변경
    left_on='event_time',
    right_on='event_time_complete',
    by=['user_id', 'content_id', 'lesson_id'],
    direction='forward',  # enter보다 크거나 같은 complete 중 가장 가까운 것 선택
    suffixes=('_enter', '_complete')
)

matched_df.head()

# # merge_asof() 실행
# matched_df = pd.merge_asof(
#     enter_df,
#     complete_df,
#     on='event_time',
#     by=['user_id', 'content_id', 'lesson_id'],
#     direction='forward',  # enter보다 크거나 같은 complete 중 가장 가까운 것 선택
#     suffixes=('_enter', '_complete')
# )

# print("\n매칭된 데이터 확인:")
# print(matched_df)

enter_df 확인:
                                  user_id              event_time event_type  \
0        28562e887d0942c8f422f1ffa1c12bc0 2022-01-01 09:30:37.101      enter   
1        459c1fe98bb5a1024f942c06c9883615 2022-01-01 10:25:19.668      enter   
2        459c1fe98bb5a1024f942c06c9883615 2022-01-01 10:28:16.287      enter   
3        459c1fe98bb5a1024f942c06c9883615 2022-01-01 10:35:04.612      enter   
4        459c1fe98bb5a1024f942c06c9883615 2022-01-01 10:43:47.037      enter   
...                                   ...                     ...        ...   
3389298  2e96e01bb82372d697a37e4e97c21dcd 2023-12-31 23:51:29.275      enter   
3389299  c2f379460037197dd941fd1fc2fc87f4 2023-12-31 23:51:53.823      enter   
3389300  1eff907fc639d1bb259dc0ce20511c01 2023-12-31 23:56:44.026      enter   
3389301  073e8f07e99a1ff2527022bed96266f2 2023-12-31 23:58:12.919      enter   
3389302  c4cd2a351011de8a8c58ea82add6fdda 2023-12-31 23:59:37.061      enter   

                          

,user_id,event_time,event_type_enter,content_id,lesson_id,event_time_complete,event_type_complete
0,28562e887d0942c8f422f1ffa1c12bc0,2022-01-01 09:30:37.101,enter,38df90d4de533d6296b52c6a1d90e26b,80aac6e426ef54f462cc4e3fea699e0c,NaT,NaN
1,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:25:19.668,enter,68a3658f1b936bfff3bd6960cd561b0a,1238493aafb242e6bb8b6891271fa8a9,2022-01-01 10:28:15.544,complete
2,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:28:16.287,enter,68a3658f1b936bfff3bd6960cd561b0a,54dd3f1d5ccc6ed8f75c99d4b92feacf,2022-01-01 10:35:04.805,complete
3,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:35:04.612,enter,68a3658f1b936bfff3bd6960cd561b0a,1ec838a253800869b1724bbb1da49376,2022-01-01 10:43:47.151,complete
4,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:43:47.037,enter,68a3658f1b936bfff3bd6960cd561b0a,50c9c2132fc891b215cf5ab9647f2476,2022-01-01 10:45:38.806,complete


In [71]:
matched_df.head(10)

,user_id,event_time,event_type_enter,content_id,lesson_id,event_time_complete,event_type_complete
0,28562e887d0942c8f422f1ffa1c12bc0,2022-01-01 09:30:37.101,enter,38df90d4de533d6296b52c6a1d90e26b,80aac6e426ef54f462cc4e3fea699e0c,NaT,NaN
1,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:25:19.668,enter,68a3658f1b936bfff3bd6960cd561b0a,1238493aafb242e6bb8b6891271fa8a9,2022-01-01 10:28:15.544,complete
2,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:28:16.287,enter,68a3658f1b936bfff3bd6960cd561b0a,54dd3f1d5ccc6ed8f75c99d4b92feacf,2022-01-01 10:35:04.805,complete
3,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:35:04.612,enter,68a3658f1b936bfff3bd6960cd561b0a,1ec838a253800869b1724bbb1da49376,2022-01-01 10:43:47.151,complete
4,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:43:47.037,enter,68a3658f1b936bfff3bd6960cd561b0a,50c9c2132fc891b215cf5ab9647f2476,2022-01-01 10:45:38.806,complete
5,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:45:39.186,enter,68a3658f1b936bfff3bd6960cd561b0a,0934eafafaf2aeeaab6d312bca7d7f2a,2022-01-01 10:47:15.614,complete
6,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:47:15.341,enter,68a3658f1b936bfff3bd6960cd561b0a,1ccbb1451cd590e160d3951be4e5593d,2022-01-01 10:49:09.078,complete
7,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:49:09.384,enter,68a3658f1b936bfff3bd6960cd561b0a,f5f843256dccdee297a878c0b38f85ad,2022-01-01 10:51:34.166,complete
8,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:51:34.356,enter,68a3658f1b936bfff3bd6960cd561b0a,3ed53ef72b4b2ea7a8379729f40a5f92,2022-01-01 10:53:59.415,complete
9,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:53:59.331,enter,68a3658f1b936bfff3bd6960cd561b0a,70f508c48ababa25afbe3cbea7ea3f49,2022-01-01 10:57:47.110,complete


In [74]:
print(matched_df['user_id'].nunique())
print(matched_df['event_time'].nunique())
print(matched_df['event_time'].nunique())
print(matched_df['event_type_enter'].value_counts())
print(matched_df['event_type_complete'].value_counts())

12860
3389155
3389155
event_type_enter
enter    3389303
Name: count, dtype: int64
event_type_complete
complete    1865160
Name: count, dtype: int64


In [75]:
len(matched_df)

3389303

In [79]:
matched_df['event_time_complete'].describe()

count                          1865160
mean     2023-02-18 16:15:58.624575488
min         2022-01-01 10:28:15.544000
25%      2022-10-04 23:27:12.250500096
50%         2023-03-10 20:10:21.172000
75%      2023-07-15 20:39:44.827000064
max         2023-12-31 23:57:19.262000
Name: event_time_complete, dtype: object

In [80]:
matched_df['event_time_complete'].isna().sum()

1524143

In [78]:
first_paid_complete_lesson_finalised_unique['event_time'].describe()

count                          1860818
mean     2023-03-01 15:18:34.779366912
min         2022-01-01 10:17:51.086000
25%      2022-09-26 15:30:23.270500096
50%      2023-03-09 11:15:06.221499904
75%      2023-08-07 14:51:43.572750080
max         2023-12-31 23:59:51.635000
Name: event_time, dtype: object

In [77]:
matched_df['event_type_enter'].describe()

count     3389303
unique          1
top         enter
freq      3389303
Name: event_type_enter, dtype: object

In [81]:
matched_df[matched_df['event_time_complete'].duplicated()]

,user_id,event_time,event_type_enter,content_id,lesson_id,event_time_complete,event_type_complete
14,dffa3c95fccc9fb9d341dec064c7b390,2022-01-01 12:11:29.416,enter,101e1d0dcc38d9c86156f008a145083e,dc1ebb970edca58235ac1315cf5b4efc,NaT,NaN
15,dffa3c95fccc9fb9d341dec064c7b390,2022-01-01 12:14:30.221,enter,101e1d0dcc38d9c86156f008a145083e,e1853746c4dc666795b5c69676cc3bd1,2022-01-01 12:18:08.869,complete
17,dffa3c95fccc9fb9d341dec064c7b390,2022-01-01 12:19:34.171,enter,101e1d0dcc38d9c86156f008a145083e,87600c3743298dc45b0c878e7b287461,2022-01-01 12:24:07.896,complete
19,4c15107f42687b928341da3a62fd78a6,2022-01-01 12:28:58.210,enter,68a3658f1b936bfff3bd6960cd561b0a,277a60cfe001aa2d666cc37c25b88dc4,NaT,NaN
20,4c15107f42687b928341da3a62fd78a6,2022-01-01 12:29:45.504,enter,68a3658f1b936bfff3bd6960cd561b0a,b6d8e9e372cf4f12984572698db58739,NaT,NaN
...,...,...,...,...,...,...,...
3389296,7b0e13e15554512df3c5cec123dcea6d,2023-12-31 23:49:15.764,enter,634f93f037bb2c534c2f99fdd16a04f0,bf5757d9fd7fbb7f8fbc1e2657326aaf,NaT,NaN
3389298,2e96e01bb82372d697a37e4e97c21dcd,2023-12-31 23:51:29.275,enter,c8e53577758bf66711094352a5abd8c3,dda87259e23f07283671639769cf8c41,NaT,NaN
3389299,c2f379460037197dd941fd1fc2fc87f4,2023-12-31 23:51:53.823,enter,61b6463287573f00de13a930805a52d6,09c8a2d29087cb3d47823acb5f2705e3,NaT,NaN
3389301,073e8f07e99a1ff2527022bed96266f2,2023-12-31 23:58:12.919,enter,c8e53577758bf66711094352a5abd8c3,501ca50ff5f76d8a619aec225022d76d,NaT,NaN


In [82]:
matched_df[matched_df['event_time_complete'] == '2022-01-01 12:18:08.869']

,user_id,event_time,event_type_enter,content_id,lesson_id,event_time_complete,event_type_complete
13,dffa3c95fccc9fb9d341dec064c7b390,2022-01-01 12:02:09.333,enter,101e1d0dcc38d9c86156f008a145083e,e1853746c4dc666795b5c69676cc3bd1,2022-01-01 12:18:08.869,complete
15,dffa3c95fccc9fb9d341dec064c7b390,2022-01-01 12:14:30.221,enter,101e1d0dcc38d9c86156f008a145083e,e1853746c4dc666795b5c69676cc3bd1,2022-01-01 12:18:08.869,complete


In [83]:
# NaN 값을 제외하고 중복된 'event_time_complete' 값 찾기
duplicates_without_na = matched_df.dropna(subset=['event_time_complete'])

# 중복된 값만 필터링
duplicates = duplicates_without_na[duplicates_without_na['event_time_complete'].duplicated()]

# 중복된 값의 개수 출력
print(f"중복된 값 개수: {duplicates.shape[0]}")

중복된 값 개수: 831223


In [89]:
# 'user_id', 'lesson_id', 'event_time_complete' 기준으로 중복된 행 찾기
duplicates_1 = matched_df.dropna(subset=['event_time_complete'])
duplicates_1 = duplicates_1[duplicates_1.duplicated(subset=['user_id', 'lesson_id', 'event_time_complete'], keep=False)]
duplicates_1

,user_id,event_time,event_type_enter,content_id,lesson_id,event_time_complete,event_type_complete
12,459c1fe98bb5a1024f942c06c9883615,2022-01-01 11:01:39.394,enter,68a3658f1b936bfff3bd6960cd561b0a,b6d8e9e372cf4f12984572698db58739,2022-01-01 12:32:17.921,complete
13,dffa3c95fccc9fb9d341dec064c7b390,2022-01-01 12:02:09.333,enter,101e1d0dcc38d9c86156f008a145083e,e1853746c4dc666795b5c69676cc3bd1,2022-01-01 12:18:08.869,complete
15,dffa3c95fccc9fb9d341dec064c7b390,2022-01-01 12:14:30.221,enter,101e1d0dcc38d9c86156f008a145083e,e1853746c4dc666795b5c69676cc3bd1,2022-01-01 12:18:08.869,complete
16,dffa3c95fccc9fb9d341dec064c7b390,2022-01-01 12:18:11.636,enter,101e1d0dcc38d9c86156f008a145083e,87600c3743298dc45b0c878e7b287461,2022-01-01 12:24:07.896,complete
17,dffa3c95fccc9fb9d341dec064c7b390,2022-01-01 12:19:34.171,enter,101e1d0dcc38d9c86156f008a145083e,87600c3743298dc45b0c878e7b287461,2022-01-01 12:24:07.896,complete
...,...,...,...,...,...,...,...
3389246,2e96e01bb82372d697a37e4e97c21dcd,2023-12-31 22:49:40.971,enter,c8e53577758bf66711094352a5abd8c3,31b6d029bf28623ae7f2f21598383a78,2023-12-31 23:04:46.196,complete
3389248,2e96e01bb82372d697a37e4e97c21dcd,2023-12-31 22:50:13.949,enter,c8e53577758bf66711094352a5abd8c3,31b6d029bf28623ae7f2f21598383a78,2023-12-31 23:04:46.196,complete
3389270,0f638fda4145317d92c06ae775caea10,2023-12-31 23:09:49.854,enter,c8e53577758bf66711094352a5abd8c3,d2bb89959260e45f5e2b6dc0207bc3e3,2023-12-31 23:11:54.723,complete
3389273,b62b06f8611b42c5319a96a63efe3146,2023-12-31 23:15:59.692,enter,03b4a290c08f7fa83933827a994f2e61,d3bb803194ae048d7d2bbb8b72b547e1,2023-12-31 23:20:38.810,complete


In [91]:
# matched_df에서 'user_id', 'lesson_id', 'event_time_complete' 기준으로 중복된 행이 있다면, 그 중 첫 번째 행만 event_time_complete을 남기고 나머지는 NAN으로 변경
matched_df.loc[duplicates_1.index, 'event_time_complete'] = None

In [92]:
matched_df.head(10)

,user_id,event_time,event_type_enter,content_id,lesson_id,event_time_complete,event_type_complete
0,28562e887d0942c8f422f1ffa1c12bc0,2022-01-01 09:30:37.101,enter,38df90d4de533d6296b52c6a1d90e26b,80aac6e426ef54f462cc4e3fea699e0c,NaT,NaN
1,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:25:19.668,enter,68a3658f1b936bfff3bd6960cd561b0a,1238493aafb242e6bb8b6891271fa8a9,2022-01-01 10:28:15.544,complete
2,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:28:16.287,enter,68a3658f1b936bfff3bd6960cd561b0a,54dd3f1d5ccc6ed8f75c99d4b92feacf,2022-01-01 10:35:04.805,complete
3,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:35:04.612,enter,68a3658f1b936bfff3bd6960cd561b0a,1ec838a253800869b1724bbb1da49376,2022-01-01 10:43:47.151,complete
4,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:43:47.037,enter,68a3658f1b936bfff3bd6960cd561b0a,50c9c2132fc891b215cf5ab9647f2476,2022-01-01 10:45:38.806,complete
5,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:45:39.186,enter,68a3658f1b936bfff3bd6960cd561b0a,0934eafafaf2aeeaab6d312bca7d7f2a,2022-01-01 10:47:15.614,complete
6,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:47:15.341,enter,68a3658f1b936bfff3bd6960cd561b0a,1ccbb1451cd590e160d3951be4e5593d,2022-01-01 10:49:09.078,complete
7,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:49:09.384,enter,68a3658f1b936bfff3bd6960cd561b0a,f5f843256dccdee297a878c0b38f85ad,2022-01-01 10:51:34.166,complete
8,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:51:34.356,enter,68a3658f1b936bfff3bd6960cd561b0a,3ed53ef72b4b2ea7a8379729f40a5f92,2022-01-01 10:53:59.415,complete
9,459c1fe98bb5a1024f942c06c9883615,2022-01-01 10:53:59.331,enter,68a3658f1b936bfff3bd6960cd561b0a,70f508c48ababa25afbe3cbea7ea3f49,2022-01-01 10:57:47.110,complete


In [ ]:
# event_time_complete - event_time 이 0인 경우도 삭제해야겠지?
matched_df['time_diff'] = matched_df['event_time_complete'] - matched_df['event_time']
matched_df['time_diff'] = matched_df['time_diff'].dt.total_seconds()
matched_df[matched_df['time_diff'] == 0]

,user_id,event_time,event_type_enter,content_id,lesson_id,event_time_complete,event_type_complete,time_diff
252927,b5d34abdbe05af0aa06ee355eb23f6d1,2022-05-06 12:43:50.574,enter,db1819d3a7a4c4d85dccb3b9e982fbcd,09c94cdc1ac94c79a93f3f89fe7e4660,2022-05-06 12:43:50.574,complete,0.0
266937,ec6f3f9ecbfd86897a214d340aafbea6,2022-05-10 20:52:44.178,enter,afe201c04b426366e0f8e9e2207c78c2,e600af4dc248233d2e6a9b7f060d47ed,2022-05-10 20:52:44.178,complete,0.0
616055,9664c2d20e147f3d95df289f5435d564,2022-08-08 19:57:30.433,enter,7cebec6e79cc981dbf421c5ec2aedd9d,a7ccb5424862396df12d7438a971d68d,2022-08-08 19:57:30.433,complete,0.0
616817,048f1cd57d7dc4bfe118c391844eab53,2022-08-08 22:11:18.939,enter,7cebec6e79cc981dbf421c5ec2aedd9d,1dd275b54ba262b8488a3a9b1bae1e9e,2022-08-08 22:11:18.939,complete,0.0
621365,5edf0171807dc3d6cda014b65052b989,2022-08-09 18:43:43.710,enter,5fc64d786416b980cfda86afd69e4516,38fe9d687a405928bfb240c722bc4456,2022-08-09 18:43:43.710,complete,0.0
...,...,...,...,...,...,...,...,...
3314948,bc71d3b6dd9bdb6e400de221805e7ad8,2023-11-12 23:02:07.348,enter,61b6463287573f00de13a930805a52d6,0cf175a1781a8e0020bf6c6bdc80e78a,2023-11-12 23:02:07.348,complete,0.0
3330944,0b3f816f3fca3737099841d7769c7c96,2023-11-15 17:21:01.021,enter,165def8885b8a2ec0c92404048d7e345,eca2fdba58a6d41b586b22bd2bcb78bf,2023-11-15 17:21:01.021,complete,0.0
3336360,517d5b50f8f7d5617bd81e8b24436ad5,2023-11-16 17:44:46.617,enter,61b6463287573f00de13a930805a52d6,eb40ff18855f20f7cd2d55dbc48b49ae,2023-11-16 17:44:46.617,complete,0.0
3339361,2b0f3deb662b30366c39fc59410727e0,2023-11-17 10:45:53.609,enter,393f678469f63d90f30618f68acae498,3441ada6ad0d8528dc4cad6d92a68e77,2023-11-17 10:45:53.609,complete,0.0
